1. Introduction
==

In an earlier mission, we learned about train/test validation, a simple technique for testing a machine learning model's accuracy on new data that the model wasn't trained on. In this mission, we'll focus on more robust techniques.

To start, we'll focus on the **holdout validation** technique, which involves:

- splitting the full dataset into 2 partitions:
    - a training set
    - a test set
- training the model on the training set,
- using the trained model to predict labels on the test set,
- computing an error metric to understand the model's effectiveness,
- switch the training and test sets and repeat,
- average the errors.

In holdout validation, we usually use a 50/50 split instead of the 75/25 split from train/test validation. This way, we remove number of observations as a potential source of variation in our model performance.

<img width="600" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1Nwq2puuGhziyQ82eukrPctHQ8UGJ93Vt">

Let's start by splitting the data set into 2 nearly equivalent halves.

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**: 

- Use the <span style="background-color: #F9EBEA; color:##C0392B">numpy.random.permutation()</span> function to shuffle the ordering of the rows in <span style="background-color: #F9EBEA; color:##C0392B">dc_listings</span>.
- Select the first 1862 rows and assign to <span style="background-color: #F9EBEA; color:##C0392B">split_ones</span>.
- Select the remaining 1861 rows and assign to <span style="background-color: #F9EBEA; color:##C0392B">split_two</span>.


In [88]:
import numpy as np
import pandas as pd

dc_listings = pd.read_csv("dc_airbnb.csv")
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

In [89]:
np.random.seed(1)

dc_listings = dc_listings.loc[np.random.permutation(dc_listings.index)]

split_one = dc_listings.iloc[:1862]
split_two = dc_listings.iloc[1862:]

2. Holdout Validation
==

Now that we've split our data set into 2 dataframes, let's:

- train a k-nearest neighbors model on the first half,
- test this model on the second half,
- train a k-nearest neighbors model on the second half,
- test this model on the first half.

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**: 

- Train a k-nearest neighbors model using the default algorithm (<span style="background-color: #F9EBEA; color:##C0392B">auto</span>) and the default number of neighbors (<span style="background-color: #F9EBEA; color:##C0392B">5</span>) that:
    - Uses the <span style="background-color: #F9EBEA; color:##C0392B">accommodates</span> column from <span style="background-color: #F9EBEA; color:##C0392B">train_one</span> for training and
    - Tests it on  <span style="background-color: #F9EBEA; color:##C0392B">test_one</span>.
- Assign the resulting RMSE value to <span style="background-color: #F9EBEA; color:##C0392B">iteration_one_rmse</span>.
- Train a k-nearest neighbors model using the default algorithm (<span style="background-color: #F9EBEA; color:##C0392B">auto</span>) and the default number of neighbors (<span style="background-color: #F9EBEA; color:##C0392B">5</span>) that:
    - Uses the <span style="background-color: #F9EBEA; color:##C0392B">accommodates</span> column from <span style="background-color: #F9EBEA; color:##C0392B">train_two</span> for training and
    - Tests it on  <span style="background-color: #F9EBEA; color:##C0392B">test_two</span>.
- Assign the resulting RMSE value to <span style="background-color: #F9EBEA; color:##C0392B">iteration_two_rmse</span>.
- Use <span style="background-color: #F9EBEA; color:##C0392B">numpy.mean()</span> to calculate the average of the 2 RMSE values and assign to <span style="background-color: #F9EBEA; color:##C0392B">avg_rmse</span>.

In [20]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

knn = KNeighborsRegressor(n_neighbors = 5, algorithm = 'auto')

knn.fit(train_one[['accommodates']], train_one['price'])
iteration_one_rmse = np.sqrt(mean_squared_error(knn.predict(test_one[['accommodates']]), test_one['price']))

knn.fit(train_two[['accommodates']], train_two['price'])
iteration_two_rmse = np.sqrt(mean_squared_error(knn.predict(test_two[['accommodates']]), test_two['price']))
                          
avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])

In [21]:
avg_rmse

128.96254732948216

3. K-Fold Cross Validation
==

If we average the two RMSE values from the last step, we get an RMSE value of approximately **132.10**. <span style="background-color: #F9EBEA; color:##C0392B">Holdout validation</span> is actually a specific example of a larger class of validation techniques called **k-fold cross-validation**. While holdout validation is better than train/test validation because the model isn't repeatedly biased towards a specific subset of the data, both models that are trained only use half the available data. K-fold cross validation, on the other hand, takes advantage of a larger proportion of the data during training while still rotating through different subsets of the data to avoid the issues of train/test validation.

Here's the algorithm from k-fold cross validation:

- splitting the full dataset into **k** equal length partitions,
    - selecting **k-1** partitions as the training set and
    - selecting the remaining partition as the test set
- training the model on the training set,
- using the trained model to predict labels on the test fold,
- computing the test fold's error metric,
- repeating all of the above steps **k-1** times, until each partition has been used as the test set for an iteration,
- calculating the mean of the **k** error values.

Holdout validation is essentially a version of k-fold cross validation when **k** is equal to **2**. Generally, **5** or **10** folds is used for k-fold cross-validation. Here's a diagram describing each iteration of 5-fold cross validation:

<img width="600" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1i9sScUbJqDLeCtd3InrBbaeHXI7vkAyQ">


As you increase the number the folds, the number of observations in each fold decreases and the variance of the fold-by-fold errors increases. Let's start by manually partitioning the data set into 5 folds. Instead of splitting into 5 dataframes, let's add a column that specifies which fold the row belongs to. 

Tip:
>```python
# change the value of column 'fold' to 1 in the first three rows
dc_listings.set_value(dc_listings.index[0:3], "fold", 1)
```

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**:

1. Add a new column to <span style="background-color: #F9EBEA; color:##C0392B">dc_listings</span> named <span style="background-color: #F9EBEA; color:##C0392B">fold</span> that contains the fold number each row belongs to:
    - Fold **1** should have rows from index **0 to 744**, including both of those rows.
    - Fold **2** should have rows from index **744 to 1488**, including both of those rows.
    - Fold **3** should have rows from index **1488 to 2232**, including both of those rows.
    - Fold **4** should have rows from index **2232 to 2976**, including both of those rows.
    - Fold **5** should have rows from index **2976 to 3723**, including both of these rows.
2. Display the unique value counts for the fold column to confirm that each fold has roughly the same number of elements.

In [90]:
dc_listings = dc_listings.assign(fold=int(0))

In [91]:
dc_listings.reset_index(inplace=True)
dc_listings.loc[:744, 'fold'] = 1
dc_listings.loc[744:1488,'fold'] = 2
dc_listings.loc[1488:2233,'fold'] = 3
dc_listings.loc[2232:2976,'fold'] = 4
dc_listings.loc[2976:,'fold'] = 5

4. First iteration
==

Let's start by performing the first iteration of k-fold cross validation on a simple, univariate model.

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**:

- Train a k-nearest neighbors model using the <span style="background-color: #F9EBEA; color:##C0392B">accommodates</span> column as the sole feature from folds <span style="background-color: #F9EBEA; color:##C0392B">2</span> to <span style="background-color: #F9EBEA; color:##C0392B">5</span> as the training set.
    - tip:
>```python
a = [2,3]
df[df.name_column.isin(a)]
```
- Use the model to make predictions on the test set (<span style="background-color: #F9EBEA;
color:##C0392B">accommodates</span> column from fold **1**) and assign the predicted labels to <span style="background-color: #F9EBEA; color:##C0392B">labels</span>.
- Calculate the RMSE value by comparing the <span style="background-color: #F9EBEA; color:##C0392B">price</span> column with the predicted <span style="background-color: #F9EBEA; color:##C0392B">labels</span>.
- Assign the RMSE value to <span style="background-color: #F9EBEA; color:##C0392B">iteration_one_rmse</span>.

In [29]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

knn = KNeighborsRegressor(n_neighbors = 5, algorithm = 'auto')

knn.fit(dc_listings[dc_listings.fold.isin([2, 5])][['accommodates']], dc_listings[dc_listings.fold.isin([2, 5])]['price'])
labels = knn.predict(dc_listings[dc_listings.fold.isin([1])][['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(labels, dc_listings[dc_listings.fold.isin([1])][['price']]))

iteration_one_rmse

191.79146134682463

5. Function for training models
==

From the first iteration, we achieved an RMSE value of **125.99**. While this is one of the lowest RMSE values we achieved in the last few missions, let's calculate the RMSE values for the remaining iterations. To make the iteration process easier, let's wrap the code we wrote in the previous screen in a function.

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**:

1. Write a function named <span style="background-color: #F9EBEA; color:##C0392B">train_and_validate</span> that takes in a dataframe as the first parameter (<span style="background-color: #F9EBEA; color:##C0392B">df</span>) and a list of fold values (<span style="background-color: #F9EBEA; color:##C0392B">1</span> to <span style="background-color: #F9EBEA; color:##C0392B">5</span> in our case) as the second parameter (<span style="background-color: #F9EBEA; color:##C0392B">folds</span>). This function should:
    - Train <span style="background-color: #F9EBEA; color:##C0392B">n</span> models (where <span style="background-color: #F9EBEA; color:##C0392B">n</span> is number of folds) and perform k-fold cross validation (using <span style="background-color: #F9EBEA; color:##C0392B">n</span> folds). Use the default <span style="background-color: #F9EBEA; color:##C0392B">k</span> value for the <span style="background-color: #F9EBEA; color:##C0392B">KNeighborsRegressor</span> class.
    - Return a list of RMSE values, where the first element is the RMSE for when fold <span style="background-color: #F9EBEA; color:##C0392B">1</span> was the test set, the second element is the RMSE for when fold <span style="background-color: #F9EBEA; color:##C0392B">2</span> was the test set, and so on.
2. Use the <span style="background-color: #F9EBEA; color:##C0392B">train_and_validate</span> function to return the list of RMSE values for the <span style="background-color: #F9EBEA; color:##C0392B">dc_listings</span> Dataframe and assign to <span style="background-color: #F9EBEA; color:##C0392B">rmses</span>.
3. Calculate the mean of these values and assign to <span style="background-color: #F9EBEA; color:##C0392B">avg_rmse</span>.
4. Display both <span style="background-color: #F9EBEA; color:##C0392B">rmses</span> and <span style="background-color: #F9EBEA; color:##C0392B">avg_rmse</span>.


In [97]:
# Use np.mean to calculate the mean.
import numpy as np
fold_ids = [1,2,3,4,5]

def train_and_validate(df, folds):
    knn = KNeighborsRegressor()
    rmse_values = []
    for i in range(1,len(folds)+1):
        train_folds = list(fold_ids)
        train_folds.remove(i)
        test_folds = i
        knn.fit(dc_listings[dc_listings.fold.isin(train_folds)][['accommodates']], dc_listings[dc_listings.fold.isin(train_folds)]['price'])
        labels = knn.predict(dc_listings[dc_listings.fold==i][['accommodates']])
        rmse_values.append(np.sqrt(mean_squared_error(labels, dc_listings[dc_listings.fold==i]['price'])))
        
    return rmse_values

rmses = train_and_validate(dc_listings, fold_ids)
avg_rmse = np.mean(rmses)
print(rmses)
print(avg_rmse)

[105.06498501055819, 140.31953569184228, 153.25636421383291, 108.30319349137453, 171.39722950247619]
135.668261582


6. Performing K-Fold Cross Validation Using Scikit-Learn
==

While the average RMSE value was approximately **129.56**, the RMSE values ranged from **123.46** all the way to **138.72**. This large amount of variability between the RMSE values means that we're either using a poor model or a poor evaluation criteria (or a bit of both!). By implementing your own k-fold cross-validation function, you hopefully acquired a good understanding of the inner workings of the technique. The function we wrote, however, has many limitations. If we want to now change the number of folds we want to use, we need to make the function more general so it can also handle randomizing the ordering of the rows in the dataframe and splitting into folds.

In machine learning, we're interested in building a good model and accurately understand how well it will perform. To build a better k-nearest neighbors model, we can change the features it uses or tweak the number of neighbors (a hyperparameter). To accurately understand a model's performance, we can perform k-fold cross validation and select the proper number of folds. We've learned how scikit-learn makes it easy for us to quickly experiment with these different knobs when it comes to building a better model. Let's now dive into how we can use scikit-learn to handle cross-validation as well.


First, we instantiate an instance of the [KFold class](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) from  <span style="background-color: #F9EBEA; color:##C0392B">sklearn.model_selection</span>:

>```python
from sklearn.model_selection import KFold
kf = KFold(n_folds, shuffle=False, random_state=None)
```

where:

- <span style="background-color: #F9EBEA; color:##C0392B">n_folds</span> is the number of folds you want to use,
- <span style="background-color: #F9EBEA; color:##C0392B">shuffle</span> is used to toggle shuffling of the ordering of the observations in the dataset,
- <span style="background-color: #F9EBEA; color:##C0392B">random_state</span> is used to specify the random seed value if <span style="background-color: #F9EBEA; color:##C0392B">shuffle</span> is set to <span style="background-color: #F9EBEA; color:##C0392B">True</span>.


You'll notice here that no parameters depend on the data set at all. This is because the KFold class returns an iterator object which we use in conjunction with the [cross_val_score() function](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), also from <span style="background-color: #F9EBEA; color:##C0392B">sklearn.model_selection</span>. Together, these 2 functions allow us to compactly train and test using k-fold cross validation:

Here are the relevant parameters for the <span style="background-color: #F9EBEA; color:##C0392B">cross_val_score</span> function:

>```python
from sklearn.model_selection import cross_val_score
cross_val_score(estimator, X, Y, scoring=None, cv=None)
```

where:

- <span style="background-color: #F9EBEA; color:##C0392B">estimator</span> is a sklearn model that implements the <span style="background-color: #F9EBEA; color:##C0392B">fit</span> method (e.g. instance of <span style="background-color: #F9EBEA; color:##C0392B">KNeighborsRegressor</span>),
-  <span style="background-color: #F9EBEA; color:##C0392B">X</span> is the list or 2D array containing the features you want to train on,
- <span style="background-color: #F9EBEA; color:##C0392B">Y</span> is a list containing the values you want to predict (target column),
- <span style="background-color: #F9EBEA; color:##C0392B">scoring</span> is a string describing the scoring criteria (list of accepted values [here](http://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)).
    - Some evaluation metrics (like mean squared error) are naturally descending scores (the smallest score is best) and as such are reported as negative by the  <span style="background-color: #F9EBEA; color:##C0392B">cross_val_score()</span> function. This is important to note, because some scores will be reported as negative that by definition can never be negative.
- <span style="background-color: #F9EBEA; color:##C0392B">cv</span> describes the number of folds. Here are some examples of accepted values:
    - an instance of the <span style="background-color: #F9EBEA; color:##C0392B">KFold</span> class,
    - an integer representing the number of folds.
    
Depending on the scoring criteria you specify, either a single total value is returned one value for each fold. Here's the general workflow for performing k-fold cross-validation using the classes we just described:

- instantiate the scikit-learn model class you want to fit,
- instantiate the <span style="background-color: #F9EBEA; color:##C0392B">KFold</span> class and using the parameters to specify the k-fold cross-validation attributes you want,
- use the <span style="background-color: #F9EBEA; color:##C0392B">cross_val_score()</span> function to return the scoring metric you're interested in.

<br>
<div class="alert alert-success">
<b>Guided Exercise.</b>
</div>


In [62]:
from sklearn.model_selection import cross_val_score, KFold

# knn model
model = KNeighborsRegressor()

# kfold instance
kf = KFold(5, shuffle=True, random_state=1)

# cross validation (knn,x,y,scoring,kfold)
mses = cross_val_score(model, dc_listings[["accommodates"]], 
                       dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)

# root mean squared error
rmses = np.sqrt(np.absolute(mses))

# average error
avg_rmse = np.mean(rmses)

print(rmses)
print(avg_rmse)

[  96.25083561   98.2398227   124.87238307  227.23391352  168.38087793]
142.995566567


7. Exploring Different K Values
==

Choosing the right <span style="background-color: #F9EBEA; color:##C0392B">k</span> value when performing k-fold cross validation is more of an art and less of a science. As we discussed earlier in the mission, a <span style="background-color: #F9EBEA; color:##C0392B">k</span> value of <span style="background-color: #F9EBEA; color:##C0392B">2</span> is really just holdout validation. On the other end, setting <span style="background-color: #F9EBEA; color:##C0392B">k</span> equal to **n** (the number of observations in the data set) is known as **leave-one-out cross validation**, or **LOOCV** for short. Through lots of trial and error, data scientists have converged on <span style="background-color: #F9EBEA; color:##C0392B">10</span> as the standard <span style="background-color: #F9EBEA; color:##C0392B">k</span> value.

In the following code block, we display the results of varying <span style="background-color: #F9EBEA; color:##C0392B">k</span> from **3 to 23**. For each <span style="background-color: #F9EBEA; color:##C0392B">k</span> value, we calculate and display the average RMSE value across all of the folds and the standard deviation of the RMSE values. Across the many different <span style="background-color: #F9EBEA; color:##C0392B">k</span> values, it seems like the average RMSE value is around <span style="background-color: #F9EBEA; color:##C0392B">128</span>. 
<br>
<div class="alert alert-success">
<b>Guided Exercise.</b>
</div>


In [61]:
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]],
                           dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print(str(fold), "folds: ", "avg RMSE: ", str(avg_rmse), "std RMSE: ", str(std_rmse))

3 folds:  avg RMSE:  130.241113525 std RMSE:  33.8224588753
5 folds:  avg RMSE:  142.995566567 std RMSE:  49.511835738
7 folds:  avg RMSE:  138.798092396 std RMSE:  43.9755423505
9 folds:  avg RMSE:  126.785310282 std RMSE:  36.1877009487
10 folds:  avg RMSE:  131.817845321 std RMSE:  34.3853202147
11 folds:  avg RMSE:  128.520346424 std RMSE:  37.1935930162
13 folds:  avg RMSE:  122.197051186 std RMSE:  41.2948951891
15 folds:  avg RMSE:  129.832684903 std RMSE:  43.4306418425
17 folds:  avg RMSE:  131.421694504 std RMSE:  43.0233643077
19 folds:  avg RMSE:  126.726422355 std RMSE:  48.8562172175
21 folds:  avg RMSE:  123.156701843 std RMSE:  49.4378544412
23 folds:  avg RMSE:  122.696083298 std RMSE:  43.145630879


8. Bias-Variance Tradeoff
==

So far, we've been working under the assumption that a lower RMSE always means that a model is more accurate. This isn't the complete picture, unfortunately. A model has two sources of error, **bias** and **variance**.

Bias describes error that results in bad assumptions about the learning algorithm. For example, assuming that only one feature, like a car's weight, relates to a car's fuel efficiency will lead you to fit a simple, univariate regression model that will result in high bias. The error rate will be high since a car's fuel efficiency is affected by many other factors besides just its weight.

Variance describes error that occurs because of the variability of a model's predicted values. If we were given a dataset with 1000 features on each car and used every single feature to train an incredibly complicated multivariate regression model, we will have low bias but high variance. In an ideal world, we want low bias and low variance but in reality, there's always a tradeoff.

The standard deviation of the RMSE values can be a proxy for a model's **variance** while the average RMSE is a proxy for a model's **bias**. Bias and variance are the 2 observable sources of error in a model that we can indirectly control.


<img width="600" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1Ws58dUyjBEmDPzqZA6vRmShScFZxoeEV">


While k-nearest negihbors can make predictions, it isn't a mathematical model. A mathematical model is usually an equation that can exist without the original data, which isn't true with k-nearest neighbors. In the next classes, we'll learn about a mathematical model called linear regression. We'll explore the bias-variance tradeoff in greater depth in these next 2 courses because of its importance when working with mathematical models in particular.
